In [1]:
# import pandas for data wrangling
import pandas as pd


# import numpy for Scientific computations
import numpy as np


# import machine learning libraries
import xgboost as xgb
from sklearn.metrics import accuracy_score


# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [2]:
df = pd.read_csv('Data/dataframe.csv', index_col = 0)
df.index = pd.to_datetime(df.index)
df = df.drop(columns = ["Festivo_Regional", "Humedad_Relativa", "Precipitacion", "Radiacion", "Velocidad_Viento"])

df.loc[:, "lag_24"] = df.Spot_electricidad.shift(24)
df.loc[:, "lag_48"] = df.Spot_electricidad.shift(48)
df.loc[:, "lag_1_semana"] = df.Spot_electricidad.shift(24*7)

In [3]:
from hyperopt import hp
import numpy as np
from sklearn.metrics import mean_squared_error


# XGB parameters
xgb_reg_params = {
    'learning_rate':    hp.choice('learning_rate',    np.arange(0.05, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth',        np.arange(5, 16, 1, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 8, 1, dtype=int)),
    'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.3, 0.8, 0.1)),
    'subsample':        hp.uniform('subsample', 0.8, 1),
    'n_estimators':     100,
}
xgb_fit_params = {
    'eval_metric': 'rmse',
    'early_stopping_rounds': 10,
    'verbose': False
}
xgb_para = dict()
xgb_para['reg_params'] = xgb_reg_params
xgb_para['fit_params'] = xgb_fit_params
xgb_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))


# LightGBM parameters
lgb_reg_params = {
    'learning_rate':    hp.choice('learning_rate',    np.arange(0.05, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth',        np.arange(5, 16, 1, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 8, 1, dtype=int)),
    'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.3, 0.8, 0.1)),
    'subsample':        hp.uniform('subsample', 0.8, 1),
    'n_estimators':     100,
}
lgb_fit_params = {
    'eval_metric': 'l2',
    'early_stopping_rounds': 10
}
lgb_para = dict()
lgb_para['reg_params'] = lgb_reg_params
lgb_para['fit_params'] = lgb_fit_params
lgb_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))


# CatBoost parameters
ctb_reg_params = {
    'learning_rate':     hp.choice('learning_rate',     np.arange(0.05, 0.31, 0.05)),
    'max_depth':         hp.choice('max_depth',         np.arange(5, 16, 1, dtype=int)),
    'colsample_bylevel': hp.choice('colsample_bylevel', np.arange(0.3, 0.8, 0.1)),
    'n_estimators':      100,
    'eval_metric':       'RMSE',
}
ctb_fit_params = {
    'early_stopping_rounds': 10,
    'verbose': False
}
ctb_para = dict()
ctb_para['reg_params'] = ctb_reg_params
ctb_para['fit_params'] = ctb_fit_params
ctb_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))

import lightgbm as lgb
import xgboost as xgb
import catboost as ctb
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials


class HPOpt(object):

    def __init__(self, x_train, x_test, y_train, y_test):
        self.x_train = x_train
        self.x_test  = x_test
        self.y_train = y_train
        self.y_test  = y_test

    def process(self, fn_name, space, trials, algo, max_evals):
        fn = getattr(self, fn_name)
        try:
            result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
        except Exception as e:
            return {'status': STATUS_FAIL,
                    'exception': str(e)}
        return result, trials

    def xgb_reg(self, para):
        reg = xgb.XGBRegressor(**para['reg_params'])
        return self.train_reg(reg, para)

    def lgb_reg(self, para):
        reg = lgb.LGBMRegressor(**para['reg_params'])
        return self.train_reg(reg, para)

    def ctb_reg(self, para):
        reg = ctb.CatBoostRegressor(**para['reg_params'])
        return self.train_reg(reg, para)

    def train_reg(self, reg, para):
        reg.fit(self.x_train, self.y_train,
                eval_set=[(self.x_train, self.y_train), (self.x_test, self.y_test)],
                **para['fit_params'])
        pred = reg.predict(self.x_test)
        loss = para['loss_func'](self.y_test, pred)
        return {'loss': loss, 'status': STATUS_OK}

In [4]:
X = df.dropna()
y = X.pop('Spot_electricidad')

In [5]:
x_train = X.loc[:"2020"]
x_test = X.loc["2021":"2021-06"]

y_train = y.loc[:"2020"]
y_test = y.loc["2021":"2021-06"]

In [6]:
obj = HPOpt(x_train, x_test, y_train, y_test)

xgb_opt = obj.process(fn_name='xgb_reg', space=xgb_para, trials=Trials(), algo=tpe.suggest, max_evals=100)
lgb_opt = obj.process(fn_name='lgb_reg', space=lgb_para, trials=Trials(), algo=tpe.suggest, max_evals=100)
ctb_opt = obj.process(fn_name='ctb_reg', space=ctb_para, trials=Trials(), algo=tpe.suggest, max_evals=100)


100%|███████████████████████████████████████████████| 100/100 [02:23<00:00,  1.43s/trial, best loss: 16.24740327951033]
[1]	training's l2: 100.472	valid_1's l2: 704.035                                                                       
[2]	training's l2: 64.8467	valid_1's l2: 573.872                                                                       
[3]	training's l2: 45.6341	valid_1's l2: 491.676                                                                       
[4]	training's l2: 29.9718	valid_1's l2: 427.277                                                                       
[5]	training's l2: 21.592	valid_1's l2: 387.823                                                                        
[6]	training's l2: 17.0494	valid_1's l2: 369.694                                                                       
[7]	training's l2: 14.3995	valid_1's l2: 356.145                                                                       
[8]	training's l2: 12.6303	valid_1's l2:

C:\Users\carlo\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "



[19]	training's l2: 7.41098	valid_1's l2: 324.583                                                                      
[20]	training's l2: 7.2168	valid_1's l2: 326.413                                                                       
[21]	training's l2: 7.05208	valid_1's l2: 319.432                                                                      
[22]	training's l2: 6.89389	valid_1's l2: 318.865                                                                      
[23]	training's l2: 6.67935	valid_1's l2: 318.988                                                                      
[24]	training's l2: 6.52805	valid_1's l2: 318.519                                                                      
[25]	training's l2: 6.3688	valid_1's l2: 318.29                                                                        
[26]	training's l2: 6.22952	valid_1's l2: 320.247                                                                      
[27]	training's l2: 6.11895	valid_1's l2

C:\Users\carlo\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "



[20]	training's l2: 18.3332	valid_1's l2: 368.245                                                                      
[21]	training's l2: 16.9049	valid_1's l2: 362.04                                                                       
[22]	training's l2: 15.7773	valid_1's l2: 355.929                                                                      
[23]	training's l2: 15.1321	valid_1's l2: 347.718                                                                      
[24]	training's l2: 14.4183	valid_1's l2: 341.537                                                                      
[25]	training's l2: 14.078	valid_1's l2: 335.649                                                                       
[26]	training's l2: 13.5329	valid_1's l2: 331.077                                                                      
[27]	training's l2: 12.8101	valid_1's l2: 326.247                                                                      
[28]	training's l2: 12.3694	valid_1's l2

...

[19]	training's l2: 7.98406	valid_1's l2: 290.74
[20]	training's l2: 7.70972	valid_1's l2: 293.15                                                                       
[21]	training's l2: 7.52798	valid_1's l2: 293.465                                                                      
[22]	training's l2: 7.36642	valid_1's l2: 293.332                                                                      
[23]	training's l2: 7.17286	valid_1's l2: 295.855                                                                      
[24]	training's l2: 7.00337	valid_1's l2: 296.267                                                                      
[25]	training's l2: 6.8973	valid_1's l2: 296.802                                                                       
[26]	training's l2: 6.76486	valid_1's l2: 295.527                                                                      
[27]	training's l2: 6.65773	valid_1's l2: 293.926                                                              

C:\Users\carlo\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "



[20]	training's l2: 7.58625	valid_1's l2: 295.359
[21]	training's l2: 7.39834	valid_1's l2: 295.489                                                                      
[22]	training's l2: 7.262	valid_1's l2: 294.649                                                                        
[23]	training's l2: 7.04579	valid_1's l2: 296.485                                                                      
[24]	training's l2: 6.88394	valid_1's l2: 296.139                                                                      
[25]	training's l2: 6.76401	valid_1's l2: 295.547                                                                      
[26]	training's l2: 6.62342	valid_1's l2: 296.472                                                                      
[1]	training's l2: 115.499	valid_1's l2: 753.777                                                                       
[2]	training's l2: 82.6517	valid_1's l2: 604.739                                                              

C:\Users\carlo\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "



[15]	training's l2: 10.0473	valid_1's l2: 296.375                                                                      
[16]	training's l2: 9.66296	valid_1's l2: 294.636                                                                      
[17]	training's l2: 9.44264	valid_1's l2: 293                                                                          
[18]	training's l2: 9.18042	valid_1's l2: 287.944                                                                      
[19]	training's l2: 8.9878	valid_1's l2: 287.147                                                                       
[20]	training's l2: 8.60233	valid_1's l2: 287.134                                                                      
[21]	training's l2: 8.30591	valid_1's l2: 285.287                                                                      
[22]	training's l2: 8.09538	valid_1's l2: 286.323                                                                      
[23]	training's l2: 7.91992	valid_1's l2

In [9]:
xgb_opt

({'colsample_bytree': 0,
  'learning_rate': 4,
  'max_depth': 0,
  'min_child_weight': 0,
  'subsample': 0.9359717435350037},
 <hyperopt.base.Trials at 0x2853c5bb310>)